# Notebook Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
pd.options.display.float_format = '{:.10f}'.format

# Quality assurance

In [ ]:
# 4hr = 2004
# 1hr = 8015
# 30m = 5856
# 5m = 35136
# 1m = 175680
years = [2023]
intervals = ["4hr"]
# lengths = [2004, 8015, 5856, 35136, 175680]
master_coin = sorted(['ADA', 'AVAX', 'BNB', 'BTC', 'DAI', 'DOT', 'ETH', 'LINK', 'LTC', 'MATIC', 'SOL', 'TRX', 'USDT', 'XLM', 'XRP'])
attached_coin = sorted(['ADA', 'AVAX', 'BNB', 'BTC', 'DAI', 'DOT', 'ETH', 'LINK', 'LTC', 'MATIC', 'SOL', 'TRX', 'USDT', 'XLM', 'XRP'])
all_coins = []

y = 0
i = 0
mc = 0
while y < len(years):
  print("___________________"+str(years[y])+"___________________ \n")

  while i < len(intervals):

    while mc < len(master_coin):
      print("Starting check for {}-{}-{}...".format(master_coin[mc], years[y], intervals[i]))
      ac = 0

      while ac < len(attached_coin):

        frames = []

        # Creating 1:1 dataframe for same coin path
        if master_coin[mc] == attached_coin[ac]:
          same = pd.DataFrame(index=np.arange(2004), columns=np.arange(11))
          same.columns = ["unix", "open", "high", "low", "close", "volume", "close_time", "quote_volume", "count", "tb_volume", "tb_quote_volume"]
          same.columns.values[0] = "{}{} unix".format(master_coin[mc], attached_coin[ac])
          same["open"] = 1
          same["high"] = 1
          same["low"] = 1
          same["close"] = 1
          all_coins.append(same)
          ac += 1

        else:
          try:
            # Fetching existing CSV files
            dir = "/content/drive/MyDrive/crypto/{}/{}/{}/{}{}/".format(years[y], intervals[i], master_coin[mc], master_coin[mc], attached_coin[ac])
            fileslist = os.listdir(dir)
            for file in sorted(fileslist):
              frames.append(pd.read_csv(dir+file, header=None))

            # Converting CSV file to a pandas DataFrame
            master = pd.concat(frames)

            # Transforming DataFrame format
            master.columns = ["unix", "open", "high", "low", "close", "volume", "close_time", "quote_volume", "count", "tb_volume", "tb_quote_volume", "ignore"]
            master.columns.values[0] = "{}{} unix".format(master_coin[mc], attached_coin[ac])
            master["close_time"] = master["close_time"].astype(str)
            master.reset_index(drop=True, inplace=True)
            master.drop(columns="ignore", inplace = True)

            # Storing to list to concat all DataFrames later
            all_coins.append(master)
            ac += 1

          except:
            try:
              # Creating reverse path
              dir = "/content/drive/MyDrive/crypto/{}/{}/{}/{}{}/".format(years[y], intervals[i], attached_coin[ac], attached_coin[ac], master_coin[mc])
              fileslist = os.listdir(dir)
              for file in sorted(fileslist):
                frames.append(pd.read_csv(dir+file, header=None))

              master = pd.concat(frames)
              master.columns = ["unix", "open", "high", "low", "close", "volume", "close_time", "quote_volume", "count", "tb_volume", "tb_quote_volume", "ignore"]
              master.columns.values[0] = "{}{} unix".format(master_coin[mc], attached_coin[ac])
              master["close_time"] = master["close_time"].astype(str)
              master["open"] = 1/master["open"]
              master["high"] = 1/master["high"]
              master["low"] = 1/master["low"]
              master["close"] = 1/master["close"]
              master["volume"] = np.nan
              master["quote_volume"] = np.nan
              master["count"] = np.nan
              master["tb_volume"] = np.nan
              master["tb_quote_volume"] = np.nan
              master.drop(columns="ignore", inplace = True)
              master.reset_index(drop=True, inplace=True)

              all_coins.append(master)
              ac += 1

            except:
              # Creating blank dataframe for coins that are not available on K-line
              missing = pd.DataFrame(index=np.arange(2004), columns=np.arange(11))
              missing.columns = ["unix", "open", "high", "low", "close", "volume", "close_time", "quote_volume", "count", "tb_volume", "tb_quote_volume"]
              missing.columns.values[0] = "{}{} unix".format(master_coin[mc], attached_coin[ac])
              all_coins.append(missing)
              print("Note: path {}{} is not available. 11 columns are created and filled with nulls".format(master_coin[mc], attached_coin[ac]))
              ac += 1
              pass

      # Checking whether all unix timestamps are same per row
      df = pd.concat(all_coins, axis=1, ignore_index = False)
      unix = df.iloc[:, ::11]
      existing_unix = []
      non_null_cols = unix.notnull().all()
      for u in range(0, len(non_null_cols)):
        if non_null_cols[u] == True:
          existing_unix.append(non_null_cols.index[u])
          check = df[existing_unix].eq(df[existing_unix].iloc[:, 0], axis=0).all(1)

      if check.value_counts().index[0] == True and check.value_counts()[0] == len(df):
        print("All unix timestamps match!".format(years[y], intervals[i]))

        # Fill in all unix NAs
        for u in range(0, 15):
          if df.filter(like="unix", axis=1).isnull().all().value_counts()[0] == 15:
            pass

          else:
            if df.filter(like="unix", axis=1).isnull().all()[u] == True:
              df[df.filter(like="unix", axis=1).isnull().all().index[u]] = df[existing_unix[0]]

            # Fill in all close time NAs
            close_time = df.filter(like="close_time", axis=1)
            non_null_cols = close_time.notnull().all()
            for ct in range(0, len(non_null_cols)):
              if non_null_cols[ct] == True:
                existing_ct = close_time.iloc[:, ct]

            for ct in range(0, 15):
              if df.filter(like="close_time", axis=1).isnull().all()[ct] == True:
                df[df.filter(like="close_time", axis=1).isnull().all().index[ct]] = existing_ct

        dir = "/content/drive/MyDrive/crypto/master-files/{}".format(master_coin[mc])
        fname = "{}-{}-{}.csv".format(master_coin[mc], years[y], intervals[i])
        df.to_csv(dir+fname, float_format='%.10f', index=False)
        print("Master file for {}-{}-{} is downloaded to ".format(master_coin[mc], years[y], intervals[i])+dir+fname+"\n")
      else:
        print("WARNING: Mismatching unix occured! \n")
        dir = "/content/drive/MyDrive/crypto/master-files/{}".format(master_coin[mc])
        fname = "{}-{}-{}.csv".format(master_coin[mc], years[y], intervals[i])
        df.to_csv(dir+fname, float_format='%.10f', index=False)
        print("Master file for {}-{}-{} is downloaded to ".format(master_coin[mc], years[y], intervals[i])+dir+fname+"\n")

      del master
      all_coins = []
      mc += 1
    i += 1
  y += 1
  i = 0

___________________2023___________________ 

Starting check for ADA-2023-4hr...
Note: path ADAAVAX is not available. 11 columns are created and filled with nulls
Note: path ADADAI is not available. 11 columns are created and filled with nulls
Note: path ADADOT is not available. 11 columns are created and filled with nulls
Note: path ADALINK is not available. 11 columns are created and filled with nulls
Note: path ADALTC is not available. 11 columns are created and filled with nulls
Note: path ADAMATIC is not available. 11 columns are created and filled with nulls
Note: path ADASOL is not available. 11 columns are created and filled with nulls
Note: path ADATRX is not available. 11 columns are created and filled with nulls
Note: path ADAXLM is not available. 11 columns are created and filled with nulls
Note: path ADAXRP is not available. 11 columns are created and filled with nulls
All unix timestamps match!
Master file for ADA-2023-4hr is downloaded to /content/drive/MyDrive/crypto/ADA

In [3]:
test = pd.read_csv("/content/drive/MyDrive/crypto/master-files/BNB/BNB-2023-30m.csv")
test.iloc[:,::11]

,BNBADA unix,BNBAVAX unix,BNBBNB unix,BNBBTC unix,BNBDAI unix,BNBDOT unix,BNBETH unix,BNBLINK unix,BNBLTC unix,BNBMATIC unix,BNBSOL unix,BNBTRX unix,BNBUSDT unix,BNBXLM unix,BNBXRP unix
0,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000,1690848000000
1,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000,1690849800000
2,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000,1690851600000
3,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000,1690853400000
4,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000,1690855200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5851,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000,1701379800000
5852,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000,1701381600000
5853,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000,1701383400000
5854,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000,1701385200000
